In [1]:
import elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
import networkx as nx

import pandas as pd
import numpy as np
import time

import multiprocessing as mp

es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])
#res = Search(using=es, index="filebeat*").query("match", username="root")
res = Search(using=es, index="packetbeat*")\
        .query('bool', filter=Q('exists', field='ip')\
               & Q('exists', field='client_ip')\
               & Q('exists', field='type') & Q('exists', field='status')\
               & Q('exists', field='bytes_in') & Q('exists', field='bytes_out')\
               & Q('match', type='ICMP')
              )

response = res.execute()

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

Filtered logs: 119442
Total number of logs: 119442


In [2]:
#start_time = time.time()
#df2 = pd.DataFrame(((hit["@timestamp"],hit['type'],hit['status'],hit['ip'],hit['client_ip'],hit['bytes_in'],hit['bytes_out']) for hit in res),\
#                    columns=['Timestamp','type','status','ip','client_ip','bytes_in','bytes_out'])
start_time = time.time()

results= [[hit["@timestamp"],hit['type'],hit['status'],hit['ip'],hit['client_ip'],hit['bytes_in'],hit['bytes_out']] for hit in res]

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0016312599182128906 seconds ---


In [3]:
len(results)

10

In [4]:
print(results[0:10])

[['2018-10-24T21:43:50.052Z', 'icmp', 'OK', '8.8.8.8', '10.1.1.65', 56, 56], ['2018-10-24T21:43:41.035Z', 'icmp', 'OK', '8.8.8.8', '10.1.1.65', 56, 56], ['2018-10-24T21:43:48.048Z', 'icmp', 'OK', '8.8.8.8', '10.1.1.65', 56, 56], ['2018-10-24T21:44:12.089Z', 'icmp', 'OK', '8.8.8.8', '10.1.1.65', 56, 56], ['2018-10-24T21:44:14.093Z', 'icmp', 'OK', '8.8.8.8', '10.1.1.65', 56, 56], ['2018-10-24T21:44:02.071Z', 'icmp', 'OK', '8.8.8.8', '10.1.1.65', 56, 56], ['2018-10-24T21:44:13.091Z', 'icmp', 'OK', '8.8.8.8', '10.1.1.65', 56, 56], ['2018-10-24T21:47:14.135Z', 'icmp', 'OK', '10.1.1.4', '10.1.1.65', 56, 56], ['2018-10-24T22:27:16.377Z', 'icmp', 'OK', '10.1.1.61', '203.178.148.19', 4, 4], ['2018-10-24T22:28:04.232Z', 'icmp', 'OK', '10.1.1.48', '206.117.25.90', 4, 4]]


In [5]:
#"/home/ubuntu/logs/cups_access.log"
#def process(es_query):
#    results= [[hit["@timestamp"],hit['type'],hit['status'],hit['ip'],hit['client_ip'],hit['bytes_in'],hit['bytes_out']] for hit in es_query]
#    for i in results:
#        print(i)
#    return results
             
#init objects
#cores=47
#pool = mp.Pool(cores)

#start_time = time.time()
#print("============== RESULTS =============")
#pool.map(process,res)

#print(netflows_lst[0:10])

#clean up
#pool.close()

#print("--- %s seconds ---" % (time.time() - start_time))



In [7]:
df_icmp = pd.DataFrame(((hit["@timestamp"],hit['type'],hit['status'],hit['ip'],hit['client_ip'],hit['bytes_in'],hit['bytes_out']) for hit in res),\
                    columns=['Timestamp','type','status','ip','client_ip','bytes_in','bytes_out'])
df_icmp.to_pickle("icmp_dataframe.pkl")